The nandc boot image is the main boot partition which holds the kernel and 

In [1]:
# The partition size of the image in bytes

!du tablet/nandc.boot.img

16388	tablet/nandc.boot.img


In [2]:
# We can use file to find out what kind of image we have.

!file tablet/nandc.boot.img

tablet/nandc.boot.img: Android bootimg, kernel (0x40008000), ramdisk (0x41000000), page size: 2048, cmdline (console=ttyS0,115200 rw init=/init loglevel=4)


In [1]:
# We can unpack this type of image file using one of the tools from the android sdk

!tools/unpackbootimg -i tablet/nandc.boot.img -o tablet/boot/

BOARD_KERNEL_CMDLINE console=ttyS0,115200 rw init=/init loglevel=4
BOARD_KERNEL_BASE 40000000
BOARD_PAGE_SIZE 2048


In [2]:
# And this is what we get from unpacking this image file

!ls -lgG tablet/boot/

total 9234
drwxrwxrwx 1    4096 Mar 17 09:36 img-ramdisk
-rwxrwxrwx 1       9 Mar 20 20:52 nandc.boot.img-base
-rwxrwxrwx 1      46 Mar 20 20:52 nandc.boot.img-cmdline
-rwxrwxrwx 1       5 Mar 20 20:52 nandc.boot.img-pagesize
-rwxrwxrwx 1 1808423 Mar 20 20:52 nandc.boot.img-ramdisk.gz
-rwxrwxrwx 1 7637412 Mar 20 20:52 nandc.boot.img-zImage
-rwxrwxrwx 1      62 Mar 16 13:55 README.md


In [3]:
%%bash
# Lets create a directory and unpack the ramdisk file

cd tablet/boot/img-ramdisk/
gunzip -c ../nandc.boot.img-ramdisk.gz | cpio -i

8272 blocks


In [4]:
# And what we have here

!ls -lgG tablet/boot/img-ramdisk/

total 3994
drwxrwxrwx 1       0 Mar 16 14:08 data
-rwxrwxrwx 1     116 Mar 20 20:52 default.prop
drwxrwxrwx 1       0 Mar 16 14:08 dev
-rwxrwxrwx 1     851 Mar 20 20:52 fstab.sun8i
-rwxrwxrwx 1  113536 Mar 20 20:52 init
-rwxrwxrwx 1    2487 Mar 20 20:52 init.goldfish.rc
-rwxrwxrwx 1 2457600 Mar 20 20:52 initlogo.rle
-rwxrwxrwx 1   20824 Mar 20 20:52 init.rc
-rwxrwxrwx 1     132 Mar 20 20:52 init.recovery.sun8i.rc
-rwxrwxrwx 1    6557 Mar 20 20:52 init.sun8i.rc
-rwxrwxrwx 1    3481 Mar 20 20:52 init.sun8i.usb.rc
-rwxrwxrwx 1    1795 Mar 20 20:52 init.trace.rc
-rwxrwxrwx 1    3947 Mar 20 20:52 init.usb.rc
-rwxrwxrwx 1 1453290 Mar 20 20:52 nand.ko
drwxrwxrwx 1       0 Mar 16 14:08 proc
-rwxrwxrwx 1      58 Mar 16 13:59 README.md
drwxrwxrwx 1     344 Mar 20 20:52 sbin
drwxrwxrwx 1       0 Mar 16 14:08 sys
drwxrwxrwx 1       0 Mar 16 14:08 system
-rwxrwxrwx 1     272 Mar 20 20:52 ueventd.goldfish.rc
-rwxrwxrwx 1    4024 Mar 20 20:52 ueventd.rc
-rwxrwxrwx 1     995 Mar 20 20:52 ueventd.sun8i

In [5]:
!cat tablet/boot/img-ramdisk/init.rc

# Copyright (C) 2012 The Android Open Source Project
#
# IMPORTANT: Do not create world writable files or directories.
# This is a common source of Android security bugs.
#

import /init.usb.rc
import /init.${ro.hardware}.rc
import /init.trace.rc

on early-init
    # Set init and its forked children's oom_adj.
    write /proc/1/oom_adj -16

    # Set the security context for the init process.
    # This should occur before anything else (e.g. ueventd) is started.
    setcon u:r:init:s0

    start ueventd

# create mountpoints
    mkdir /mnt 0775 root system

on init

sysclktz 0

loglevel 6

# setup the global environment
    export PATH /sbin:/vendor/bin:/system/sbin:/system/bin:/system/xbin
    export LD_LIBRARY_PATH /vendor/lib:/system/lib
    export ANDROID_BOOTLOGO 1
    export ANDROID_ROOT /system
    export ANDROID_ASSETS /system/app
    export ANDROID_DATA /data
    export ANDROID_STORAGE /storage
    export ASEC_MOUNTPOINT /mnt/asec
    export LOOP_MOUNTPOINT /mnt/obb
    expor